## Installation des dépendances

Exécutez cette cellule en premier pour installer tous les packages nécessaires. Cette opération peut prendre quelques minutes.

In [1]:
!pip install langchain langchain-community langchain-text-splitters sentence-transformers faiss-cpu rank-bm25 pypdf2 python-dotenv

  Using cached langchain_community-0.4.1-py3-none-any.whl.metadata (3.0 kB)
Using cached langchain_community-0.4.1-py3-none-any.whl (2.5 MB)



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
# Pour langchain >=1.0.0
# from langchain.text_splitters import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.retrievers import EnsembleRetriever, BM25Retriever


ModuleNotFoundError: No module named 'langchain.text_splitter'

# Projet RAG - Système de Recherche et Génération Augmentée

Ce notebook implémente un système RAG (Retrieval-Augmented Generation) pour répondre à des questions basées sur des documents PDF de cours d'intelligence artificielle. Le système utilise des embeddings multilingues, un retriever hybride et un modèle de langage pour générer des réponses précises.

## Étape 1 : Imports et configuration

Dans cette étape, nous importons les bibliothèques nécessaires et configurons l'environnement. Nous chargeons également la clé API OpenRouter depuis un fichier .env pour éviter de l'exposer dans le code.

In [16]:
import os
from dotenv import load_dotenv
from PyPDF2 import PdfReader

# ✅ Imports corrigés pour LangChain >= 1.0.0
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain_community.retrievers import BM25Retriever
from langchain.retrievers import EnsembleRetriever

load_dotenv()

# 🔑 Clé OpenRouter
OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")
if not OPENROUTER_API_KEY:
    raise ValueError("Merci de mettre ta clé OPENROUTER_API_KEY dans le fichier .env")

c:\Users\moham\OneDrive\Bureau\Rag-firstprtjt\ai & applications prjt\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: No module named 'langchain.chains'

## Étape 2 : Chargement et extraction des textes des PDF

Ici, nous parcourons le dossier 'data/' pour lire tous les fichiers PDF. Pour chaque PDF, nous extrayons le texte de toutes les pages, en nettoyant les espaces et les sauts de ligne. Les textes sont stockés dans une liste avec des métadonnées sur la source.

In [ ]:
import pickle

import os



# Charger les documents depuis le cache si disponible

if os.path.exists("raw_documents.pkl"):

    with open("raw_documents.pkl", "rb") as f:

        raw_documents = pickle.load(f)

    print(f"Documents chargés depuis le cache : {len(raw_documents)} cours PDF.")

else:

    folder_path = "data/"  # dossier contenant tous les PDFs

    raw_documents = []

    

    for filename in os.listdir(folder_path):

        if filename.lower().endswith(".pdf"):

            pdf_path = os.path.join(folder_path, filename)

            reader = PdfReader(pdf_path)

            page_texts = []

            for page in reader.pages:

                content = (page.extract_text() or "").replace("\n", " ").strip()

                if content:

                    page_texts.append(content)

            full_text = " ".join(page_texts)

            if full_text:

                raw_documents.append({"text": full_text, "metadata": {"source": filename}})

    

    if not raw_documents:

        raise ValueError("Aucun texte extrait des PDFs. Vérifie que le dossier data/ contient des fichiers PDF lisibles.")

    

    # Sauvegarder les documents extraits

    with open("raw_documents.pkl", "wb") as f:

        pickle.dump(raw_documents, f)

    print(f"Nombre de cours PDF exploitables : {len(raw_documents)} (sauvegardé dans raw_documents.pkl)")

## Étape 3 : Découpage des textes en chunks

Les textes longs sont divisés en morceaux plus petits (chunks) pour faciliter la recherche. Nous utilisons un splitter récursif avec une taille de chunk de 350 caractères et un chevauchement de 120 caractères pour conserver le contexte entre les chunks.

In [ ]:
import pickle

import os



# Charger les chunks depuis le cache si disponible

if os.path.exists("documents_chunks.pkl"):

    with open("documents_chunks.pkl", "rb") as f:

        documents = pickle.load(f)

    print(f"Chunks chargés depuis le cache : {len(documents)} chunks.")

else:

    text_splitter = RecursiveCharacterTextSplitter(

        chunk_size=350,

        chunk_overlap=120

    )

    

    documents = []

    for item in raw_documents:

        splitted_docs = text_splitter.create_documents([item["text"]], metadatas=[item["metadata"]])

        for idx, doc in enumerate(splitted_docs):

            doc.metadata["chunk_index"] = idx

        documents.extend(splitted_docs)

    

    if not documents:

        raise ValueError("Aucun chunk généré. Vérifie les textes extraits.")

    

    # Sauvegarder les chunks

    with open("documents_chunks.pkl", "wb") as f:

        pickle.dump(documents, f)

    print(f"Total de chunks indexés : {len(documents)} (sauvegardé dans documents_chunks.pkl)")

## Étape 4 : Création des embeddings

Nous créons des représentations vectorielles (embeddings) des chunks de texte en utilisant un modèle multilingue. Ces embeddings sont normalisés pour améliorer la similarité cosinus. Nous stockons ensuite ces vecteurs dans une base de données FAISS pour une recherche rapide.

## Note importante sur la sauvegarde

Les étapes précédentes sauvegardent automatiquement les résultats intermédiaires :
- **raw_documents.pkl** : Documents extraits des PDF
- **documents_chunks.pkl** : Chunks de texte découpés
- **faiss_index/** : Index FAISS avec les embeddings

Si vous réexécutez le notebook, ces fichiers seront chargés directement depuis le disque au lieu d'être recalculés, ce qui accélère considérablement le temps d'exécution.

In [ ]:
import os



embedding_model = HuggingFaceEmbeddings(

    model_name="intfloat/multilingual-e5-large",

    encode_kwargs={"normalize_embeddings": True}

)



# Charger l'index FAISS depuis le disque si disponible

if os.path.exists("faiss_index"):

    vectorstore = FAISS.load_local("faiss_index", embedding_model, allow_dangerous_deserialization=True)

    print("Index FAISS chargé depuis le disque.")

else:

    vectorstore = FAISS.from_documents(documents, embedding_model)

    print("Index FAISS construit avec embeddings e5.")

    # Sauvegarder l'index FAISS sur le disque pour éviter de le recalculer

    vectorstore.save_local("faiss_index")

    print("Index FAISS sauvegardé dans le dossier 'faiss_index'.")

## Étape 5 : Configuration du retriever hybride

Pour améliorer la précision de la recherche, nous utilisons un retriever hybride qui combine la recherche dense (basée sur les embeddings) et la recherche sparse (BM25, basée sur les mots-clés). Le retriever dense utilise MMR pour la diversité, et nous pondérons les résultats avec 65% pour dense et 35% pour sparse.

In [ ]:
dense_retriever = vectorstore.as_retriever(
    search_type="mmr",  # Maximum Marginal Relevance pour diversité
    search_kwargs={"k": 3, "fetch_k": 4}
)

sparse_retriever = BM25Retriever.from_documents(documents)
sparse_retriever.k = 8

hybrid_retriever = EnsembleRetriever(
    retrievers=[dense_retriever, sparse_retriever],
    weights=[0.65, 0.35]
)

## Étape 6 : Configuration du modèle de langage

Nous configurons le modèle GPT-4o-mini via OpenRouter. La température est réglée à 0.4 pour des réponses plus factuelles. Nous utilisons des en-têtes personnalisés pour respecter les exigences d'OpenRouter.

In [ ]:
llm = ChatOpenAI(
    model_name="gpt-4o-mini",
    temperature=0.4,  # température plus basse pour des réponses plus factuelles
    openai_api_key=OPENROUTER_API_KEY,
    openai_api_base="https://openrouter.ai/api/v1",
    default_headers={
        "HTTP-Referer": "https://github.com/your-user/your-repo",
        "X-Title": "RAG Notebook",
    }
)

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=hybrid_retriever,
    return_source_documents=True,
    chain_type="stuff"
)

## Étape 7 : Exemple de recherche et génération de réponse

Nous testons le système avec une question sur les défis de l'intelligence artificielle. Le retriever récupère les chunks pertinents, puis le modèle génère une réponse basée sur ces informations. Les sources utilisées sont affichées pour la transparence.

In [ ]:
query = "Expliquer les principaux défis de l’intelligence artificielle"

result = qa_chain(query)

print("Réponse du modèle :")
print(result["result"])

print("\nSources utilisées :")
for i, doc in enumerate(result["source_documents"]):
    print(f"Source {i} ({doc.metadata.get('source')}, chunk {doc.metadata.get('chunk_index')})")
    print(doc.page_content[:300], "...\n")

## Étape 8 : Génération d'un quiz

Enfin, nous demandons au système de générer un quiz basé sur les documents. Cela démontre la capacité du RAG à créer du contenu nouveau et pertinent à partir des informations récupérées.

In [ ]:
query_quiz = """
Génère 5 questions de quiz avec leurs réponses à partir des passages pertinents
des cours sur l’intelligence artificielle.
"""

result_quiz = qa_chain(query_quiz)

print("Quiz généré :")
print(result_quiz["result"])